<a href="https://www.kaggle.com/code/manishkr1754/credit-card-fraud-detection?scriptVersionId=144207541" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

---
<center><h1>Credit Card Fraud Detection</h1></center>
<center><h3>Part of 30 Days 30 ML Projects Challenge</h3></center>

---

## 1) Understanding Problem Statement
---

Credit card fraud is a pervasive and costly issue in the financial industry, causing significant financial losses to both cardholders and financial institutions. Timely detection and prevention of fraudulent credit card transactions are essential for minimizing financial losses and protecting consumers. Machine learning models can be instrumental in identifying suspicious transactions and predicting the likelihood of credit card fraud based on transaction data and behavioral patterns.

The project is about **Anomaly Detection in Financial Transactions using Machine Learning**. The project falls under **Classification Machine Learning Problem**. The goal of this project is **to develop a credit card fraud detection model that can accurately identify potentially fraudulent transactions, safeguarding cardholders and financial institutions from financial losses**.

## 2) Understanding Data
---

The project uses **Credit Card Fraud Data** which contains several variables (independent variables) and the outcome variable or dependent variable.

## 3) Getting System Ready
---
Importing required libraries


In [ ]:
import numpy as np
import pandas as pd

# for model buidling
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

## 4) Data Eyeballing
---

### Laoding Data

In [ ]:
credit_card_fraud_data = pd.read_csv('Datasets/Day10_Credit_Card_Fraud_Data.csv') 

In [ ]:
credit_card_fraud_data

In [ ]:
print('The size of Dataframe is: ', credit_card_fraud_data.shape)
print('-'*100)
print('The Column Name, Record Count and Data Types are as follows: ')
credit_card_fraud_data.info()
print('-'*100)

In [ ]:
# Defining numerical & categorical columns
numeric_features = [feature for feature in credit_card_fraud_data.columns if credit_card_fraud_data[feature].dtype != 'O']
categorical_features = [feature for feature in credit_card_fraud_data.columns if credit_card_fraud_data[feature].dtype == 'O']

# print columns
print('We have {} numerical features : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features : {}'.format(len(categorical_features), categorical_features))

In [ ]:
print('Missing Value Presence in different columns of DataFrame are as follows : ')
print('-'*100)
total=credit_card_fraud_data.isnull().sum().sort_values(ascending=False)
percent=(credit_card_fraud_data.isnull().sum()/credit_card_fraud_data.isnull().count()*100).sort_values(ascending=False)
pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
print('Summary Statistics of numerical features for DataFrame are as follows:')
print('-'*100)
credit_card_fraud_data.describe()

In [ ]:
credit_card_fraud_data['Class'].value_counts()

#### `Class` is the Target Variable. Seeing the distribution of `legit transaction(Class=0)` and `fraudulent transactions(Class=1)`, the dataset is highly unbalanced.

## 4) Data Cleaning and Preprocessing
---

### Separating the data as per `Class` column for analysis

In [ ]:
legit = credit_card_fraud_data[credit_card_fraud_data.Class == 0]
fraud = credit_card_fraud_data[credit_card_fraud_data.Class == 1]

In [ ]:
legit.shape,fraud.shape

#### Statistical Look at Amount column for legit and fraud transaction

In [ ]:
legit.Amount.describe()

In [ ]:
fraud.Amount.describe()

In [ ]:
credit_card_fraud_data.groupby('Class').mean()

#### Under Sampling - Build a sample dataset containing similar distribution of legit transactions and fraud transaction, Here as per fraud transaction count 492

In [ ]:
legit_sample = legit.sample(n=492)

In [ ]:
legit_sample

#### Concatenating dataframe from Under Sample Data

In [ ]:
sampled_data = pd.concat([legit_sample, fraud], axis=0)

In [ ]:
sampled_data

In [ ]:
sampled_data['Class'].value_counts()

In [ ]:
sampled_data.groupby('Class').mean()

## 5) Model Building
---

### Creating Feature Matrix (Independent Variables) & Target Variable (Dependent Variable)

In [ ]:
# separating the data and labels
X = sampled_data.drop(columns = ['Class'], axis=1) # Feature matrix
y = sampled_data['Class'] # Target variable

In [ ]:
X

In [ ]:
y

### Data Standardization

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(X)

In [ ]:
standardized_data = scaler.transform(X)

In [ ]:
standardized_data

In [ ]:
X = standardized_data

In [ ]:
X

### Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=45)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

In [ ]:
print(y.shape, y_train.shape, y_test.shape)

### Model Comparison : Training & Evaluation

In [ ]:
models = [LogisticRegression, SVC, DecisionTreeClassifier, RandomForestClassifier]
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for model in models:
    classifier = model().fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred))
    recall_scores.append(recall_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))

In [ ]:
classification_metrics_df = pd.DataFrame({
    "Model": ["Logistic Regression", "SVM", "Decision Tree", "Random Forest"],
    "Accuracy": accuracy_scores,
    "Precision": precision_scores,
    "Recall": recall_scores,
    "F1 Score": f1_scores
})

classification_metrics_df.set_index('Model', inplace=True)
classification_metrics_df

### Inference

In the context of credit card fraud detection, 
- Logistic Regression and Support Vector Machine (SVM) exhibit strong overall performance with high accuracy, precision, and F1 scores, indicating their effectiveness in correctly identifying fraudulent transactions and minimizing false positives. While Decision Tree shows a slightly lower accuracy, its balanced precision and recall suggest it's a reasonable choice. Random Forest also performs well but falls slightly short of Logistic Regression and SVM. In real-world credit card fraud detection, prioritizing precision to reduce false positives is crucial to minimize the financial burden on customers. Thus, Logistic Regression or SVM may be preferred, depending on specific operational requirements.